In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import sys
import os

# Adiciona o caminho da raiz do projeto (BINARYCLASSIFICATION) ao sys.path
sys.path.append(os.path.abspath(".."))


In [20]:
df = pd.read_csv("../data/class_imbalance/dataset_38_sick.csv")

In [21]:
print(df.head())

    age sex on_thyroxine query_on_thyroxine on_antithyroid_medication sick  \
0  41.0   F            f                  f                         f    f   
1  23.0   F            f                  f                         f    f   
2  46.0   M            f                  f                         f    f   
3  70.0   F            t                  f                         f    f   
4  70.0   F            f                  f                         f    f   

  pregnant thyroid_surgery I131_treatment query_hypothyroid  ... TT4_measured  \
0        f               f              f                 f  ...            t   
1        f               f              f                 f  ...            t   
2        f               f              f                 f  ...            t   
3        f               f              f                 f  ...            t   
4        f               f              f                 f  ...            t   

     TT4 T4U_measured   T4U FTI_measured    

In [22]:
print(df.value_counts(df.columns[-1]))

Class
negative    3541
sick         231
Name: count, dtype: int64


In [23]:
df["Class"] = df["Class"].map({"negative": 0, "sick": 1})


In [24]:
binary_map = {"t": 1, "f": 0, "M": 1, "F": 0}
df = df.replace(binary_map)


In [25]:
df = df.replace("?", np.nan)

In [26]:
df = df.drop(columns=["referral_source"])


In [27]:
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
cat_cols = df.select_dtypes(include=['object', 'bool']).columns

# Preenche colunas numéricas com a média
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

# Preenche colunas categóricas com a moda (linha por linha, seguro)
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        moda = df[col].mode()
        if not moda.empty:
            df[col] = df[col].fillna(moda.iloc[0])


In [28]:
print("Valores ausentes restantes:", df.isnull().sum().sum())


Valores ausentes restantes: 3772


In [29]:
print(df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False))


TBG    3772
dtype: int64


In [30]:
df = df.drop(columns=["TBG"])


In [31]:
X = df.drop("Class", axis=1).values
y = df["Class"].values


In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # mantém proporção das classes
)


In [33]:
from src.logistic_regression import LogisticRegression

model = LogisticRegression(lr=0.1, max_iters=1000)
model.fit(X_train, y_train)


C:\Users\Yan\AppData\Roaming\Python\Python312\site-packages\autograd\tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
C:\Users\Yan\AppData\Roaming\Python\Python312\site-packages\autograd\numpy\numpy_vjps.py:52: RuntimeWarning: overflow encountered in square
  lambda ans, x, y : unbroadcast_f(y, lambda g: - g * x / y**2))
C:\Users\Yan\AppData\Roaming\Python\Python312\site-packages\autograd\numpy\numpy_vjps.py:74: RuntimeWarning: invalid value encountered in multiply
  defvjp(anp.exp,    lambda ans, x : lambda g: ans * g)


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred = model.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Precisão:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


Acurácia: 0.9390728476821192
Precisão: 0.0
Recall: 0.0
F1-score: 0.0
Matriz de Confusão:
[[709   0]
 [ 46   0]]


C:\Users\Yan\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
